In [1]:
import nltk
from nltk.corpus import reuters

In [2]:
categories = reuters.categories()

In [3]:
tops = {}
for cat in categories:
    tops[cat] = len(reuters.fileids(cat))

In [4]:
sorted_tops = sorted(tops.items(), key=lambda item: item[1], reverse=True)

In [5]:
sorted_tops = sorted_tops[:10]

In [7]:
sorted_tops

[('earn', 3964),
 ('acq', 2369),
 ('money-fx', 717),
 ('grain', 582),
 ('crude', 578),
 ('trade', 485),
 ('interest', 478),
 ('ship', 286),
 ('wheat', 283),
 ('corn', 237)]

In [9]:
reuters.fileids(sorted_tops[0][0])[:10]

['test/14859',
 'test/14860',
 'test/14872',
 'test/14873',
 'test/14875',
 'test/14876',
 'test/14899',
 'test/14903',
 'test/14911',
 'test/14926']

In [14]:
def build_courpse(index, percentage=0.05):
    corpus = []
    mst = reuters.fileids(sorted_tops[index][0])
    length = len(mst)
    docs = mst[: int(percentage * length)]
    for doc in docs:
        corpus.append(reuters.words(doc))
    return corpus

In [21]:
final_courpse_top_10 = []

In [61]:
labels = []
labels.extend([0]*len(build_courpse(0)))
labels.extend([1]*len(build_courpse(1, 0.1)))
for ind in range(2, 10):
    labels.extend([ind]*len(build_courpse(ind, 0.4)))

In [63]:
len(labels)

1889

In [22]:
final_courpse_top_10.extend(build_courpse(0))
final_courpse_top_10.extend(build_courpse(1, 0.1))
for ind in range(2, 10):
    final_courpse_top_10.extend(build_courpse(ind, 0.4))

In [28]:
import itertools

words = list(itertools.chain(*final_courpse_top_10))

In [44]:
# filtered_words = list(filter(lambda x: len(x)>=3, words))
final_courpse_top_10 = [" ".join(x) for x in final_courpse_top_10]

In [45]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
A = vec.fit_transform(final_courpse_top_10)

In [48]:
A = A.T

In [56]:
A = A.T

In [57]:
A.shape

(1889, 13244)

In [58]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(A)

In [59]:
kmeans.labels_.shape

(1889,)

In [64]:
real_labels = np.array(labels)

In [67]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(real_labels, kmeans.labels_)

In [14]:
def get_purity(conf_matrix, real_labels=None):
    purity = 0
    for i in range(10):
        purity += (max(conf_matrix.T[i,:]))/len(real_labels)
    print(purity)

In [131]:
get_purity(conf_matrix)

0.19904711487559554


In [15]:
def get_entropy(conf_matrix, real_labels=None):
    import math
    entropy = 0
    conf_matrix_eps = conf_matrix.T + 1.001
    for i in range(10):
        for j in range(10):
            entropy += (conf_matrix_eps[i][j] * math.log(conf_matrix_eps[i][j]/sum(conf_matrix_eps[:,i])))
    entropy *= -1 * (1/(len(real_labels)* math.log(len(real_labels))))
    print(entropy)


In [129]:
get_entropy(conf_matrix)

0.17820159785275794


In [90]:
math.log(conf_matrix_eps[1][1]/(sum(conf_matrix_eps[:,1])))

-1.747264181571106

In [60]:
kmeans.cluster_centers_.shape

(10, 13244)

In [9]:
import numpy as np
def build_courpse(index, percentage=0.05):
    corpus = []
    mst = reuters.fileids(sorted_tops[index][0])
    length = len(mst)
    docs = mst[: int(percentage * length)]
    for doc in docs:
        corpus.append(reuters.words(doc))
    return corpus
sorted_tops = sorted(tops.items(), key=lambda item: item[1], reverse=True)
sorted_tops = sorted_tops[:20]
final_courpse_top_20 = []
labels_20 = []
labels_20.extend([0]*len(build_courpse(0)))
labels_20.extend([1]*len(build_courpse(1, 0.1)))
for ind in range(2, 20):
    labels_20.extend([ind]*len(build_courpse(ind, 0.4)))
real_labels = np.array(labels_20)
for ind in range(2, 20):
    labels_20.extend([ind]*len(build_courpse(ind, 0.4)))
final_courpse_top_20.extend(build_courpse(0))
final_courpse_top_20.extend(build_courpse(1, 0.1))
for ind in range(2, 20):
    final_courpse_top_20.extend(build_courpse(ind, 0.4))
final_courpse_top_20 = [" ".join(x) for x in final_courpse_top_20]
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
A = vec.fit_transform(final_courpse_top_20)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(A)
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(real_labels, kmeans.labels_)
get_purity(conf_matrix)
get_entropy(conf_matrix)

0.05625764370158989
0.09148101634400618


In [21]:
from sklearn.decomposition import NMF
model = NMF(n_components=20, init='random', random_state=0)
W = model.fit_transform(A)
H = model.components_
conf_matrix = confusion_matrix(real_labels, np.argmax(W, axis=1))
get_purity(conf_matrix)
get_entropy(conf_matrix)

0.12922951487973908
0.1174786073578191


In [1]:
## TDT2

In [2]:
path = "/home/seyed/Downloads/TDT2_all.mat"

In [3]:
import scipy.io
mat = scipy.io.loadmat(path) 

In [6]:
A, labels = mat["fea"], mat["gnd"]

In [10]:
labels.ravel().shape

(10212,)

In [20]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=96, random_state=0).fit(A)
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(labels.ravel(), kmeans.labels_)
get_purity(conf_matrix, labels.ravel())
get_entropy(conf_matrix, labels.ravel())

0.03466509988249119
0.012642627126321452


In [19]:
len(set(labels.ravel().tolist()))

96

In [23]:
import pandas as pd 

df_label = pd.DataFrame({"label":labels.ravel()})

In [58]:
last_10 = df_label.groupby("label").size()[::-1][:10].index.to_numpy()

In [66]:
top_10 = df_label.groupby("label").size()[:10].index.to_numpy()
top_10_val = df_label.groupby("label").size()[:10].to_numpy()

In [43]:
ll = labels.ravel()

In [68]:
top_10

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint64)

In [69]:
l1 = ll.tolist()
index_top_10 = []
index_last_10 = []
for i, a in enumerate(l1):
    if a in top_10:
        index_top_10.append(i)
for i, a in enumerate(l1):
    if a in last_10:
        index_last_10.append(i)

In [70]:
index_last_10

[10201, 10202, 10203, 10204, 10205, 10206, 10207, 10208, 10209, 10210, 10211]

In [84]:
A_arr = A.toarray()
top_A , top_labels = np.zeros((len(index_top_10), A_arr.shape[1])), []
for i, j in enumerate(index_top_10):
    top_A[i,:] = A_arr[j,:]
    top_labels.append(ll[j])


In [89]:
top_labels = np.array(top_labels)

In [91]:
last_A , last_labels = np.zeros((len(index_last_10), A_arr.shape[1])), []
for i, j in enumerate(index_last_10):
    last_A[i,:] = A_arr[j,:]
    last_labels.append(ll[j])

In [92]:
last_labels = np.array(last_labels)

In [93]:
last_A.shape, last_labels.shape

((11, 36771), (11,))

In [94]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(top_A)
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(top_labels, kmeans.labels_)
get_purity(conf_matrix, top_labels)
get_entropy(conf_matrix, top_labels)

0.5021459227467812
-0.05216949623920224


In [95]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)
W = model.fit_transform(top_A)
H = model.components_
conf_matrix = confusion_matrix(top_labels, np.argmax(W, axis=1))
get_purity(conf_matrix, top_labels)
get_entropy(conf_matrix, top_labels)

0.8130364806866953
0.017263215343055787


In [96]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(last_A)
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(last_labels, kmeans.labels_)
get_purity(conf_matrix, last_labels)
get_entropy(conf_matrix, last_labels)
#-----------------------------------------------------------------
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)
W = model.fit_transform(last_A)
H = model.components_
conf_matrix = confusion_matrix(last_labels, np.argmax(W, axis=1))
get_purity(conf_matrix, last_labels)
get_entropy(conf_matrix, last_labels)

0.9090909090909093
11.368788286373869
0.9090909090909093
11.310417628880485


In [97]:
last_20 = df_label.groupby("label").size()[::-1][:20].index.to_numpy()
top_20 = df_label.groupby("label").size()[:20].index.to_numpy()

index_top_20 = []
index_last_20 = []
for i, a in enumerate(l1):
    if a in top_20:
        index_top_20.append(i)

for i, a in enumerate(l1):
    if a in last_20:
        index_last_20.append(i)

top_A_20 , top_labels_20 = np.zeros((len(index_top_20), A_arr.shape[1])), []
for i, j in enumerate(index_top_20):
    top_A_20[i,:] = A_arr[j,:]
    top_labels_20.append(ll[j])

top_labels_20 = np.array(top_labels_20)

last_A_20 , last_labels_20 = np.zeros((len(index_last_20), A_arr.shape[1])), []
for i, j in enumerate(index_last_20):
    last_A_20[i,:] = A_arr[j,:]
    last_labels_20.append(ll[j])
last_labels_20 = np.array(last_labels_20)

In [98]:
last_A_20.shape, last_labels_20.shape, top_A_20.shape , top_labels_20.shape

((42, 36771), (42,), (8741, 36771), (8741,))

In [99]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(top_A_20)
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(top_labels_20, kmeans.labels_)
get_purity(conf_matrix, top_labels_20)
get_entropy(conf_matrix, top_labels_20)
#-----------------------------------------------------------------
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)
W = model.fit_transform(top_A_20)
H = model.components_
conf_matrix = confusion_matrix(top_labels_20, np.argmax(W, axis=1))
get_purity(conf_matrix, top_labels_20)
get_entropy(conf_matrix, top_labels_20)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(last_A_20)
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(last_labels_20, kmeans.labels_)
get_purity(conf_matrix, last_labels_20)
get_entropy(conf_matrix, last_labels_20)
#-----------------------------------------------------------------
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)
W = model.fit_transform(last_A_20)
H = model.components_
conf_matrix = confusion_matrix(last_labels_20, np.argmax(W, axis=1))
get_purity(conf_matrix, last_labels_20)
get_entropy(conf_matrix, last_labels_20)